# Image Recognition Model using the cifar100 dataset

In [ ]:
# source: https://stackabuse.com/image-recognition-in-python-with-tensorflow-and-keras/ --> Tutorial used
# https://www.cs.toronto.edu/~kriz/cifar.html --> Dataset used
# https://keras.io/examples/vision/image_classification_from_scratch/ --> Keras' own tutorial on image segmentation
# https://machinelearningmastery.com/save-load-keras-deep-learning-models/ --> how to save the model


In [5]:
# https://appliedmachinelearning.blog/2018/03/24/achieving-90-accuracy-in-object-recognition-task-on-cifar-10-dataset-with-keras-convolutional-neural-networks/

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras import utils
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, Activation, Flatten, Dropout, BatchNormalization
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.datasets import cifar100
from tensorflow.keras import regularizers
from tensorflow.keras.callbacks import LearningRateScheduler
import numpy as np



In [7]:
def lr_schedule(epoch):
    lrate = 0.001
    if epoch > 75:
        lrate = 0.0005
    if epoch > 100:
        lrate = 0.0003
    return lrate

(x_train, y_train), (x_test, y_test) = cifar100.load_data()
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
 
#z-score
mean = np.mean(x_train,axis=(0,1,2,3))
std = np.std(x_train,axis=(0,1,2,3))
x_train = (x_train-mean)/(std+1e-7)
x_test = (x_test-mean)/(std+1e-7)
 
num_classes = 100
y_train = to_categorical(y_train,num_classes)
y_test = to_categorical(y_test,num_classes)
 
weight_decay = 1e-4
model = Sequential()
model.add(Conv2D(32, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay), input_shape=x_train.shape[1:]))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Conv2D(32, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))
 
model.add(Conv2D(64, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Conv2D(64, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.3))
 
model.add(Conv2D(128, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Conv2D(128, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))
 
model.add(Flatten())
model.add(Dense(num_classes, activation='softmax'))
 
model.summary()



Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 32, 32, 32)        896       
_________________________________________________________________
activation (Activation)      (None, 32, 32, 32)        0         
_________________________________________________________________
batch_normalization (BatchNo (None, 32, 32, 32)        128       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 32, 32, 32)        9248      
_________________________________________________________________
activation_1 (Activation)    (None, 32, 32, 32)        0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 32, 32, 32)        128       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 16, 16, 32)        0

In [9]:
#data augmentation
datagen = ImageDataGenerator(
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
    )
datagen.fit(x_train)
 


In [17]:
from tensorflow.keras import optimizers

#training
batch_size = 64
 
opt_rms = optimizers.RMSprop(lr=0.001,decay=1e-6)
model.compile(loss='categorical_crossentropy', optimizer=opt_rms, metrics=['accuracy'])
model.fit_generator(datagen.flow(x_train, y_train, batch_size=batch_size),\
                    steps_per_epoch=x_train.shape[0] // batch_size,epochs=125,\
                    verbose=1,validation_data=(x_test,y_test),callbacks=[LearningRateScheduler(lr_schedule)])


Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/125
781/781 [==============================] - 397s 508ms/step - loss: 4.3213 - accuracy: 0.1255 - val_loss: 3.3741 - val_accuracy: 0.2315 - lr: 0.0010
Epoch 2/125
781/781 [==============================] - 303s 387ms/step - loss: 3.2777 - accuracy: 0.2375 - val_loss: 2.8736 - val_accuracy: 0.3071 - lr: 0.0010
Epoch 3/125
781/781 [==============================] - 317s 406ms/step - loss: 2.8447 - accuracy: 0.3092 - val_loss: 2.6834 - val_accuracy: 0.3564 - lr: 0.0010
Epoch 4/125
781/781 [==============================] - 294s 377ms/step - loss: 2.6211 - accuracy: 0.3563 - val_loss: 2.4265 - val_accuracy: 0.3989 - lr: 0.0010
Epoch 5/125
781/781 [==============================] - 286s 366ms/step - loss: 2.4906 - accuracy: 0.3862 - val_loss: 2.2624 - val_accuracy: 0.4356 - lr: 0.0010
Epoch 6/125
781/781 [==============================] - 286s 366ms/step - loss: 2.3946 - accuracy: 0.4116 - val_loss: 2.1618 

781/781 [==============================] - 283s 362ms/step - loss: 1.8515 - accuracy: 0.5644 - val_loss: 1.8488 - val_accuracy: 0.5766 - lr: 0.0010
Epoch 51/125
781/781 [==============================] - 282s 361ms/step - loss: 1.8482 - accuracy: 0.5656 - val_loss: 1.9631 - val_accuracy: 0.5544 - lr: 0.0010
Epoch 52/125
781/781 [==============================] - 285s 365ms/step - loss: 1.8487 - accuracy: 0.5649 - val_loss: 1.8145 - val_accuracy: 0.5798 - lr: 0.0010
Epoch 53/125
781/781 [==============================] - 284s 364ms/step - loss: 1.8408 - accuracy: 0.5662 - val_loss: 1.8615 - val_accuracy: 0.5768 - lr: 0.0010
Epoch 54/125
781/781 [==============================] - 279s 357ms/step - loss: 1.8434 - accuracy: 0.5693 - val_loss: 2.0210 - val_accuracy: 0.5431 - lr: 0.0010
Epoch 55/125
781/781 [==============================] - 281s 360ms/step - loss: 1.8346 - accuracy: 0.5686 - val_loss: 1.9345 - val_accuracy: 0.5584 - lr: 0.0010
Epoch 56/125
781/781 [=========================

781/781 [==============================] - 316s 405ms/step - loss: 1.5605 - accuracy: 0.6241 - val_loss: 1.7265 - val_accuracy: 0.6032 - lr: 5.0000e-04
Epoch 101/125
781/781 [==============================] - 283s 363ms/step - loss: 1.5487 - accuracy: 0.6290 - val_loss: 1.7179 - val_accuracy: 0.5986 - lr: 5.0000e-04
Epoch 102/125
781/781 [==============================] - 295s 377ms/step - loss: 1.5014 - accuracy: 0.6370 - val_loss: 1.6701 - val_accuracy: 0.6100 - lr: 3.0000e-04
Epoch 103/125
781/781 [==============================] - 321s 410ms/step - loss: 1.4884 - accuracy: 0.6414 - val_loss: 1.6930 - val_accuracy: 0.6080 - lr: 3.0000e-04
Epoch 104/125
781/781 [==============================] - 298s 381ms/step - loss: 1.4670 - accuracy: 0.6475 - val_loss: 1.6438 - val_accuracy: 0.6192 - lr: 3.0000e-04
Epoch 105/125
781/781 [==============================] - 288s 368ms/step - loss: 1.4772 - accuracy: 0.6427 - val_loss: 1.6561 - val_accuracy: 0.6166 - lr: 3.0000e-04
Epoch 106/125
781/

In [18]:
#save to disk
model_json = model.to_json()
with open('model_100.json', 'w') as json_file:
    json_file.write(model_json)
model.save_weights('model_100.h5') 
 
#testing
scores = model.evaluate(x_test, y_test, batch_size=128, verbose=1)
print('\nTest result: %.3f loss: %.3f' % (scores[1]*100,scores[0]))

79/79 [==============================] - 16s 204ms/step - loss: 1.6443 - accuracy: 0.6168

Test result: 61.680 loss: 1.644


###########